# Network with Applications

In this file, we'll demonstrate the simulation of a more complicated network topology with applications. These applications will act on each node, first requesting memories to be entangled and then using these memories to teleport a qubit. The network topology, including hardware components, is shown below:

<img src="./images/star_network.png" width="800"/>

In this file, we construct the network described above and add a teleportation app to two nodes. Most of the code has been written, but some will need to be filled in. We'll be building the topology from an external json file `star_network.json`.

## Imports
We must first import the necessary tools from SeQUeNCe. For building a teleportation circuit, we'll import:

- `Circuit` is a class for describing and applying quantum circuits, with the use of the simulation kernel.

For building our specific request, we will import:

- `RequestApp` is an example application included with SeQUeNCe. We will investigate its behavior through a class that inherits from this.
- `Message` is a wrapper for classical messages to exchange between nodes.

Finally, for creating the network itself, we will import:

- `Topology` is a powerful class for creating and managing complex network topologies. We'll be using the `RouterNetTopo` subclass to build our network and intefrace with specific nodes and node types.


In [1]:
import pandas as pd
from ipywidgets import interact
import time

In [2]:
# for building teleportation functions
from sequence.components.circuit import Circuit
from numpy import sqrt

# for building application
from sequence.app.request_app import RequestApp
from sequence.message import Message

# for building network
from sequence.topology.router_net_topo import RouterNetTopo

## Teleportation Functions

Here, we define our functions for teleporting a qubit.

### Teleport 1
This function will take in the arguments

    e_pair_key: the key pointing to the state of the local entangled memory
    node: the local node from which we are teleporting a qubit

The key may be obtained from the quantum memory, as outlined later. We will then generate a qubit to teleport, and use the `run_circuit` method of the quantum circuit to teleport it. This method takes a circuit, the keys to operate on, and a measurement sample (to use for determining measurement outcomes) as arguments. We will then return the measurement results, so the entangled memory at the other node may be corrected.

### Teleport 2
This function will take in the key for the corresponding entangled memory on the receiving node, and will correct it to match the teleported qubit (and then measure it). We will take in the same arguments as the previous function, with the addition of

    x_flip: dictates if we perform an x-gate correction
    z_flip: dictates if we perform a z-gate correction

These will be passed as 0 or 1 depending on the measurement outcome of the first circuit.

In [10]:
def teleport_1(e_pair_key, node):
    # get quantum manager and measurement sample
    qm = node.timeline.quantum_manager
    meas_samp = node.get_generator().random()
    
    # generate teleported qubit
    state = (complex(sqrt(1/2)),
             complex(sqrt(1/2)))
    qubit_key = qm.new(state)
    
    # prepare teleportation circuit (TODO)
    
    # run circuit and return results (TODO)

def teleport_2(e_pair_key, node, x_flip, z_flip):
    # get quantum manager and measurement sample
    qm = node.timeline.quantum_manager
    meas_samp = node.get_generator().random()
    
    # prepare rectification circuit (TODO)
    
    # run circuit and return results (TODO)

## Custom Application

We may now define our custom application. When receiving an entangled memory, we will create our desired qubit and jointly measure it; we will then use the new `TeleportMessage` class to communicate results to the other node. The receiving node will wait whenn receiving an entangled memory. Once the classical message is sent from the originator, it will correct and measure the underlying qubit.

In [4]:
class TeleportMessage(Message):
    def __init__(self, msg_type, receiver, index: int, x_flip: int, z_flip: int):
        super().__init__(msg_type, receiver)
        self.index = index
        self.x_flip = x_flip
        self.z_flip = z_flip

class TeleportApp(RequestApp):
    def __init__(self, node, name, other_name):
        super().__init__(node)
        self.name = name
        self.other_name = other_name
        node.protocols.append(self)
        self.memos_to_measure = []
        
        self.results = [0, 0]
        
    def get_memory(self, info: "MemoryInfo") -> None:
        """Method to receive entangled memories.

        Will check if the received memory is qualified.
        If it's a qualified memory, the application sets memory to RAW state
        and release back to resource manager.
        The counter of entanglement memories, 'memory_counter', is added.
        Otherwise, the application does not modify the state of memory and
        release back to the resource manager.

        Args:
            info (MemoryInfo): info on the qualified entangled memory.
        """

        if info.state != "ENTANGLED":
            return

        if info.index in self.memo_to_reserve:
            reservation = self.memo_to_reserve[info.index]
            
            if info.remote_node == reservation.responder and info.fidelity >= reservation.fidelity:
                # we are initiator, and want to teleport qubit
                print("node {} memory {} entangled with other memory {}".format(
                    self.node.name, info.index, info.remote_memo))
                
                # run local teleportation ops (TODO)
                
                # send message to other node (TODO)
                
                # reset local memory
                self.node.resource_manager.update(None, info.memory, "RAW")
                
            elif info.remote_node == reservation.initiator and info.fidelity >= reservation.fidelity:
                # we are responder, and want to receive qubit
                # need to wait on message from sender to correct entanled memory
                self.memos_to_measure.append(info.memory)
                
    def received_message(self, src, message):
        memo_index = message.index
        
        # run local teleportation correction and save result (TODO)
        memory = self.memos_to_measure.pop(0)
        
        # reset local memory
        self.node.resource_manager.update(None, memory, "RAW")

## Building the Simulation

We'll now construct the network and add our applications. This example follows the usual process to ensure that all tools function properly:
1. Create the topology instance for the simulation to manage our network
    - This class will create a simulation timeline
    - This instantiates the Simulation Kernel (see below)
2. Create the simulated network topology. In this case, we are using an external JSON file to specify nodes and their connectivity.
    - This includes specifying hardware parameters in the `set_parameters` function, defined later
    - This instantiates the Hardware, Entanglement Management, Resource Management, and Network Management modules
3. Install custom protocols/applications and ensure all are set up properly
    - This instantiates the Application module
4. Initialize and run the simulation
5. Collect and display the desired metrics

The JSON file specifies that network nodes should be of type `QuantumRouter`, a node type defined by SeQUeNCe. This will automatically create all necessary hardware and protocol instances on the nodes, and the `Topology` class will automatically generate `BSMNode` instances on the quantum channels between such nodes.

To construct an application, we need:
- The node to attach the application to
- The name of the application instance
- The name of the node to teleport qubits to.

We can get a list of all desired application nodes, in this case routers, from the `Topology` class with the `get_nodes_by_type` method. We then set an application on each one.

In [5]:
def test(sim_time=1.5, qc_atten=1e-5):
    """
    sim_time: duration of simulation time (ms)
    qc_atten: quantum channel attenuation (dB/km)
    """
    
    network_config = "star_network.json"
    
    # here, we make a new topology using the configuration JSON file.
    # we then modify some of the simulation parameters of the network.
    network_topo = RouterNetTopo(network_config)
    set_parameters(network_topo, sim_time, qc_atten)
    
    # get two end nodes
    start_node_name = "router1"
    end_node_name = "router2"
    node1 = node2 = None

    for router in network_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        if router.name == start_node_name:
            node1 = router
        elif router.name == end_node_name:
            node2 = router
            
    start_app_name = "start_app"
    end_app_name = "end_app"
    
    # create applications (TODO)
    start_app = None
    end_app = None
        
    # run the simulation
    tl = network_topo.get_timeline()
    tl.show_progress = False
    tl.init()
    
    start_app.start(end_node_name, 1e12, 2e12, 10, 0.9)
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    print("measured |0>:", end_app.results[0])
    print("measured |1>:", end_app.results[1])

## Setting parameters

Here we define the `set_parameters` function we used earlier. This function will take a `Topology` as input and change many parameters to desired values. This will be covered in greater detail in workshop 3.

The simulation time limit will be set using the `get_timeline` method.

Quantum memories and detectors are hardware elements, and so parameters are changed by accessing the hardware included with the `QuantumRouter` and `BSMNode` node types. Many complex hardware elements, such as bsm devices or memory arrays, have methods to update parameters for all included hardware elements. This includes `update_memory_params` to change all memories in an array or `update_detector_params` to change all detectors.

We will also set the success probability and swapping degradation of the entanglement swapping protocol. This will be set in the Network management Module (specifically the reservation protocol), as this information is necessary to create and manage the rules for the Resource Management module.

Lastly, we'll update some parameters of the quantum channels. Quantum channels (and, similarly, classical channels) can be accessed from the `Topology` class as the `qchannels` field. Since these are individual hardware elements, we will set the parameters directly.

In [6]:
def set_parameters(topology, simulation_time, attenuation):
    """
    simulation_time: duration of simulation time (s)
    attenuation: attenuation on quantum channels (db/m)
    """
    
    PS_PER_S = 1e12
    
    # set timeline stop time
    topology.get_timeline().stop_time = (simulation_time * PS_PER_S)
    
    # set memory parameters
    MEMO_FREQ = 2e3
    MEMO_EXPIRE = 0
    MEMO_EFFICIENCY = 1
    MEMO_FIDELITY = 0.9349367588934053
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        memory_array = node.get_components_by_type("MemoryArray")[0]
        memory_array.update_memory_params("frequency", MEMO_FREQ)
        memory_array.update_memory_params("coherence_time", MEMO_EXPIRE)
        memory_array.update_memory_params("efficiency", MEMO_EFFICIENCY)
        memory_array.update_memory_params("raw_fidelity", MEMO_FIDELITY)

    # set detector parameters
    DETECTOR_EFFICIENCY = 0.9
    DETECTOR_COUNT_RATE = 5e7
    DETECTOR_RESOLUTION = 100
    for node in topology.get_nodes_by_type(RouterNetTopo.BSM_NODE):
        bsm = node.get_components_by_type("SingleAtomBSM")[0]
        bsm.update_detectors_params("efficiency", DETECTOR_EFFICIENCY)
        bsm.update_detectors_params("count_rate", DETECTOR_COUNT_RATE)
        bsm.update_detectors_params("time_resolution", DETECTOR_RESOLUTION)
        
    # set entanglement swapping parameters
    SWAP_SUCC_PROB = 0.90
    SWAP_DEGRADATION = 0.99
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        node.network_manager.protocol_stack[1].set_swapping_success_rate(SWAP_SUCC_PROB)
        node.network_manager.protocol_stack[1].set_swapping_degradation(SWAP_DEGRADATION)
        
    # set quantum channel parameters
    ATTENUATION = attenuation
    QC_FREQ = 1e11
    for qc in topology.qchannels:
        qc.attenuation = ATTENUATION
        qc.frequency = QC_FREQ

## Running the Simulation

All that is left is to run the simulation with user input. We'll specify:

    sim_time: duration of simulation time (s)
    qc_atten: attenuation on quantum channels (dB/m)

Note that different hardware parameters or network topologies may cause the simulation to run for a very long time.

In [11]:
interact(test, sim_time=(1, 2, 0.1), qc_atten=[0, 1e-5, 2e-5])

interactive(children=(FloatSlider(value=1.5, description='sim_time', max=2.0, min=1.0), Dropdown(description='…

<function __main__.test(sim_time=1.5, qc_atten=1e-05)>